# Particle in a Box 1D

Particle in a One-Dimensional Box (Infinite Square Well)

This notebook demonstrates how the **time-independent Schrödinger equation (TISE)**

$$
-\frac{\hbar^2}{2m}\frac{d^2\psi}{dx^2} + V(x)\Psi = E\psi
$$

reduces, inside an infinite potential well of width $L$, to a **linear eigenvalue problem** with boundary conditions
$$
\psi(0) = \psi(L) = 0.
$$

We proceed step by step, using **SymPy** to mirror each analytical operation:

1. **Formulate** the differential equation in symbolic form.  
2. **Solve** for the general wavefunction $\psi(x)$ via `dsolve`.  
3. **Apply boundary conditions** to determine integration constants $C_1, C_2$.  
4. **Construct** the corresponding homogeneous linear system and derive the **quantization condition** $\sin(kL)=0$.  
5. **Obtain** normalized eigenfunctions and discrete energy eigenvalues:
   $$
   \psi_n(x) = \sqrt{\frac{2}{L}}\sin\left(\frac{n\pi x}{L}\right), \qquad
   E_n = \frac{n^2\pi^2\hbar^2}{2mL^2}.
   $$

Throughout, the notebook maintains a one-to-one correspondence between **analytical derivation** and **symbolic computation**, so that each conceptual step in the physics can be directly inspected, executed, and extended.

---

### Learning objectives

- Connect **differential equations**, **linear algebra**, and **quantization** through symbolic computation.  
- Reinforce the physical meaning of *boundary conditions* as constraints that lead to discrete eigenvalues.  
- Demonstrate the computational structure of a **Hilbert space eigenproblem** within an operator framework.

---

> **Prerequisites:**  
> Familiarity with differential equations, basic quantum mechanics notation, and the SymPy symbolic algebra system.


In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
# imports so be able to show
from IPython.display import display, Math

# Define Variables
x = sp.Symbol('x', real=True)
L = sp.Symbol('L', positive=True, real=True)
n = sp.Symbol('n', positive=True, real=True)
m = sp.Symbol('m', positive=True, real=True)
hbar= sp.Symbol('hbar', positive=True, real=True)

E = sp.symbols('E', real=True)
k = sp.sqrt(2*m*E)/hbar
psi = sp.Function('psi')



# Defining the Schroedingr's equation


In [12]:
import sympy as sp

x = sp.Symbol('x', real=True)

class Hamiltonian1D:
    def __init__(self, m, hbar, Vx):
        """
        m, hbar: sympy symbols (mass, reduced Planck)
        Vx: sympy expression for V(x), e.g. 0 or Piecewise(...)
        """
        self.m = m
        self.hbar = hbar
        self.Vx = Vx        # V(x) as a function of x

    def kinetic(self, f_expr):
        """Kinetic term acting on wavefunction f(x)."""
        return - (self.hbar**2 / (2*self.m)) * sp.diff(f_expr, x, 2)

    def potential(self, f_expr):
        """Potential term acting on wavefunction f(x)."""
        return self.Vx * f_expr

    def H(self, f_expr):
        """Full Hamiltonian H = T + V acting on f(x)."""
        return self.kinetic(f_expr) + self.potential(f_expr)

class ParticleInABox1DAnalytical:
  def __init__(self, Vx=0):
    # Symbols
    self.x: sp.Symbol = sp.Symbol('x', real=True)
    self.L: sp.Symbol = sp.Symbol('L', positive=True, real=True)
    self.n: sp.Symbol = sp.Symbol('n', positive=True, real=True)
    self.m: sp.Symbol = sp.Symbol('m', positive=True, real=True)
    self.hbar: sp.Symbol = sp.Symbol('hbar', positive=True, real=True)
    self.E: sp.Symbol   = sp.Symbol('E', real=True)

    # Wavefunction ψ(x)
    self.psi = sp.Function('psi')(x)

    # Potential V(x) = 0 inside the well (we do the interior problem)
    assert isinstance(Vx, float) \
      or isinstance(Vx, int) \
      or isinstance(Vx, sp.Function)
    self.Vx = Vx

    # Hamiltonian operator H = T + V
    self.hamiltonian = Hamiltonian1D(self.m, self.hbar, self.Vx)

  def get_tise(self):
    """
    Time-independent Schrödinger equation H ψ = E ψ
    returns a SymPy Eq object.
    """
    lhs = self.hamiltonian.H(self.psi)      # H acting on ψ(x)
    rhs = self.E * self.psi
    return sp.Eq(lhs, rhs)


# Define the potential

In [13]:
Vx = 0.

piab1d = ParticleInABox1DAnalytical()
tise_eq = piab1d.get_tise()
print(type(tise_eq))
display(Math(sp.latex(tise_eq)))

<class 'sympy.core.relational.Equality'>


<IPython.core.display.Math object>

# General solution

In [24]:
from dataclasses import dataclass
@dataclass
class GeneralSolution():
  tise_eq: sp.Equality
  general_solution: sp.Equality

general_solution = sp.dsolve(tise_eq)
print(type(general_solution))
display(Math(sp.latex(general_solution)))

psi_general_solution = general_solution.rhs
display(Math(sp.latex(psi_general_solution)))

general_solution.rhs.free_symbols


<class 'sympy.core.relational.Equality'>


<IPython.core.display.Math object>

<IPython.core.display.Math object>

{C1, C2, E, hbar, m, x}

In [22]:
import sympy as sp
from IPython.display import display, Math

# Define Variables
x = sp.Symbol('x', positive=True, real=True)
L = sp.Symbol('L', positive=True, real=True)
n = sp.Symbol('n', positive=True, real=True)
m = sp.Symbol('m', positive=True, real=True)
hbar= sp.Symbol('hbar', positive=True, real=True)

# Schrödinger equation for infinite square well
# ψ''(x) + (2mE/ħ²) ψ(x) = 0
E = sp.Symbol('E', real=True)
k = sp.sqrt(2*m*E)/hbar
psi = sp.Function('psi')

# Solve the differential equation
tise_general_solution = sp.dsolve(tise)
display(Math(sp.latex(tise_general_solution)))

psi_general_solution = tise_general_solution.rhs
display(Math(sp.latex(psi_general_solution)))

C1, C2 = sp.symbols('C1 C2')

# Boundary conditions in algebraic form
bc1 = sp.Eq(psi_general_solution.subs(x, 0), 0)
bc2 = sp.Eq(psi_general_solution.subs(x, L), 0)
print("apply boundary conditions")
display(Math(r"\psi(0) = 0"))
display(Math(r"\Rightarrow " + sp.latex(bc1)))
display(Math(r"\psi(L) = 0"))
display(Math(r"\Rightarrow " + sp.latex(bc2)))

# Coefficient matrix acting on (C1, C2)^T
M = sp.Matrix([
    [sp.sin(0),     sp.cos(0)],
    [sp.sin(k*L),   sp.cos(k*L)]
])
display(Math(r"M = " + sp.latex(M)))

# Show the homogeneous system
display(Math(r"M \cdot \begin{bmatrix}C_1 \\ C_2\end{bmatrix} = \begin{bmatrix}0 \\ 0\end{bmatrix}"))


bc1 = sp.Eq(psi_general_solution.subs(x, 0), 0)
bc2 = sp.Eq(psi_general_solution.subs(x, L), 0)
boundary_conditions = {
    'bc1': {
      'latex':r"\psi(0) = 0",
      'sympy':sp.Eq(psi_general_solution.subs(x, 0), 0)
    },
    'bc2': {
      'latex':r"\psi(L) = 0",
      'sympy':sp.Eq(psi_general_solution.subs(x, L), 0)
    },
}

for bc_k, bc_v in boundary_conditions.items():
  display(Math(bc_v['latex']))
  display(Math(r"\Rightarrow " + sp.latex(bc_v['sympy'])))


solution_C1C2 = sp.solve(
  [v['sympy'] for v in boundary_conditions.values()],
  (C1, C2)
)

solution_C1C2
sol_C1C2 = sp.solve((bc1, bc2), (C1, C2), dict=True)
sol_C1C2
# trivial solution
# Build the coefficient matrix and determinant
M, rhs = sp.linear_eq_to_matrix(
    [bc1.lhs, bc2.lhs],
    [C1, C2]
)
display(Math(r"M = " + sp.latex(M)))
display(Math(sp.latex(rhs)))
detM = sp.simplify(M.det())
display(Math(r"\det M = " + sp.latex(detM)))

# Quantization condition: detM = 0
display(Math(r"\det M = 0 \Rightarrow " + sp.latex(sp.Eq(detM, 0))))

k_n = n*sp.pi/L
E_n = hbar**2 * k_n**2 / (2*m)
psi_n = sp.sqrt(2/L)*sp.sin(k_n*x)

display(Math(r"k_n = " + sp.latex(k_n)))
display(Math(r"E_n = " + sp.latex(E_n)))
display(Math(r"\psi_n(x) = " + sp.latex(psi_n)))




NameError: name 'tise' is not defined

## Apply boundary conditions

In [ ]:
C1, C2 = sp.symbols('C1 C2')

bc1 = sp.Eq(psi_general_solution.subs(x, 0), 0)
bc2 = sp.Eq(psi_general_solution.subs(x, L), 0)

display(Math(sp.latex(bc1)))
display(Math(sp.latex(bc2)))

sol_C1C2 = sp.solve((bc1, bc2), (C1, C2), dict=True)
sol_C1C2
# trivial solution
# Build the coefficient matrix and determinant
M = sp.Matrix([
    [sp.sin(0),     sp.cos(0)],
    [sp.sin(k*L),   sp.cos(k*L)]
])
detM = sp.simplify(M.det())
display(Math(r"\det M = " + sp.latex(detM)))

# Quantization condition: detM = 0
display(Math(r"\det M = 0 \Rightarrow " + sp.latex(sp.Eq(detM, 0))))

k_n = n*sp.pi/L
E_n = hbar**2 * k_n**2 / (2*m)
psi_n = sp.sqrt(2/L)*sp.sin(k_n*x)

display(Math(r"k_n = " + sp.latex(k_n)))
display(Math(r"E_n = " + sp.latex(E_n)))
display(Math(r"\psi_n(x) = " + sp.latex(psi_n)))


# Boundary conditions
#bc1 = sp.Eq(tise_general_solution.subs(x, 0), 0)
#bc2 = sp.Eq(tise_general_solution.subs(x, L), 0)

# Apply ψ(0)=0 → C2=0
#C2_sol = sp.solve(bc1, C2)[0]
#psi_bc1 = tise_general_solution.subs(C2, C2_sol)

# Apply ψ(L)=0 → sin(kL)=0
#k_values = sp.solve(sp.Eq(sp.sin(k*L), 0), k)
#k_values


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
# Substitute and Compute Energies

k_n = n*sp.pi/L
E_n = (hbar**2 * k_n**2) / (2*m)
psi_n = sp.sqrt(2/L) * sp.sin(k_n*x)

E_n, psi_n


(pi**2*hbar**2*n**2/(2*L**2*m), sqrt(2)*sin(pi*n*x/L)/sqrt(L))

In [ ]:
norm = sp.integrate(psi_n**2, (x, 0, L))
sp.simplify(norm)


1 - sin(2*pi*n)/(2*pi*n)